In [24]:
import csv
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing, ensemble

from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import io
from googleapiclient.http import MediaIoBaseDownload
from google.colab import files
from pydrive.drive import GoogleDrive

In [0]:
trainrawfileid = "1PJ6E2J-oFlh-GFIwkeMfVBy3mRRnzqtP"
testrawfileid = "1bI4M05xME2KC_YyJ0ogaqRZWPYiMr199"

trainfileid = "1gReV8BOf55n2uV_10UTtPACLUPrZIrsc"
testfileid = "1usmR1pG9l-dvs9VCb54vFKNrIZeW4hAg"
trainflnm = "train_clean.csv"
testflnm = "test_clean.csv"
outflnm = 'xgb_new.csv'

In [0]:
train_download = drive.CreateFile({'id': trainfileid})
test_download = drive.CreateFile({'id': testfileid})



In [0]:
train_download.GetContentFile(trainflnm)
test_download.GetContentFile(testflnm)

In [0]:
cat_cols = ["ind_empleado","sexo","ind_nuevo","indrel","indrel_1mes","tiprel_1mes",
          "indresi","indext","ind_actividad_cliente","segmento","pais_residencia","canal_entrada"]

target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']
target_cols = target_cols[2:]


In [0]:
df = pd.read_csv(trainflnm, usecols=cat_cols )


In [0]:
predictor_dict = {}
for cats_ in cat_cols:
    df_ = df.loc[df[cats_].notnull(),cats_]
    dic_ = dict(enumerate(sorted(df_.unique())))
    indic_ = {v: k for k, v in dic_.iteritems()}
    predictor_dict[cats_] = indic_

In [0]:
def getTarget(row):
	tlist = []
	for col in target_cols:
		if row[col].strip() in ['', 'NA']:
			target = 0
		else:
			target = int(float(row[col]))
		tlist.append(target)
	return tlist

def getIndex(row, col):
    val = row[col].strip()
    #strtype_ = strType(val) 
    if col in ["ind_actividad_cliente","ind_nuevo","indrel","indrel_1mes"]:
        val_ = float(val)
        valnn_ = -1
    else:
        val_ = val
        valnn_ = '0'
    ind = ""
    try:
        if val not in ['','NA']:
            ind = predictor_dict[col][val_]
        else:
            ind = valnn_ #predictor_dict[col][-99]
    except:
        print(col, val_)
        
    return ind

In [0]:
def processData(in_file_name, cust_dict):
	x_vars_list = []
	y_vars_list = []
	for row in csv.DictReader(in_file_name):
		# use only the four months as specified by breakfastpirate #
		if row['fecha_dato'] not in ['2015-05-28', '2015-06-28', '2016-05-28', '2016-06-28']:
			continue

		cust_id = int(row['ncodpers'])
		if row['fecha_dato'] in ['2015-05-28', '2016-05-28']:	
			target_list = getTarget(row)
			cust_dict[cust_id] =  target_list[:]
			continue

		x_vars = []
		for col in cat_cols:
			x_vars.append( getIndex(row, col) )
		x_vars.append(row['age']) 
		x_vars.append(row['antiguedad']) 
		x_vars.append(row['renta'])

		if row['fecha_dato'] == '2016-06-28':
			prev_target_list = cust_dict.get(cust_id, [0]*22)
			x_vars_list.append(x_vars + prev_target_list)
		elif row['fecha_dato'] == '2015-06-28':
			prev_target_list = cust_dict.get(cust_id, [0]*22)
			target_list = getTarget(row)
			new_products = [max(x1 - x2,0) for (x1, x2) in zip(target_list, prev_target_list)]
			if sum(new_products) > 0:
				for ind, prod in enumerate(new_products):
					if prod>0:
						assert len(prev_target_list) == 22
						x_vars_list.append(x_vars+prev_target_list)
						y_vars_list.append(ind)

	return x_vars_list, y_vars_list, cust_dict

In [39]:
start_time = datetime.datetime.now()
train_file =  open(trainflnm)
x_vars_list, y_vars_list, cust_dict = processData(train_file, {})
train_X = np.array(x_vars_list)
train_y = np.array(y_vars_list)
print(np.unique(train_y))
del x_vars_list, y_vars_list
train_file.close()
print(train_X.shape, train_y.shape)
print(datetime.datetime.now()-start_time)

test_file = open(testflnm)
x_vars_list, y_vars_list, cust_dict = processData(test_file, cust_dict)
test_X = np.array(x_vars_list)
del x_vars_list
test_file.close()
print(test_X.shape)
print(datetime.datetime.now()-start_time)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
((45664, 37), (45664,))
0:02:21.618570
(928621, 37)
0:02:39.883709


In [0]:
def modelfit(alg, trainX, trainY,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        print(xgb_param)
        xgtrain = xgb.DMatrix(trainX, label=trainY)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='mlogloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(trainX, trainY,eval_metric='mlogloss')
        
    #Predict training set:
    dtrain_predictions = alg.predict(trainX)
    dtrain_predprob = alg.predict_proba(trainX)
    
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(trainY, dtrain_predictions)
    print "log loss (Train): %f" % metrics.log_loss(trainY, dtrain_predprob)
                    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    return alg

In [0]:
predictors = cat_cols + ['age','antiguedad','renta']

xgbt = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=5,
 min_child_weight=1,
 num_class = 22,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective=  'multi:softprob',
 nthread=4,
 scale_pos_weight=1,
 seed=0)


In [0]:
model = modelfit(xgbt, train_X, train_y)

{'reg_alpha': 0, 'colsample_bylevel': 1, 'scale_pos_weight': 1, 'learning_rate': 0.1, 'max_delta_step': 0, 'base_score': 0.5, 'n_estimators': 200, 'seed': 0, 'num_class': 22, 'booster': 'gbtree', 'colsample_bytree': 0.8, 'silent': 1, 'missing': None, 'nthread': 4, 'min_child_weight': 1, 'subsample': 0.8, 'reg_lambda': 1, 'objective': 'multi:softprob', 'max_depth': 5, 'gamma': 0}

In [0]:
#Hyper parameter tuning

param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,num_class = 22,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=5)

gsearch1.fit(train_X,train_y)

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


In [0]:
print("Predicting..")
xgtest = xgb.DMatrix(test_X)
preds = model.predict(xgtest)
del test_X, xgtest
print(datetime.datetime.now()-start_time)

print("Getting the top products..")
target_cols = np.array(target_cols)
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)[:,:7]
test_id = np.array(pd.read_csv(testflnm, usecols=['ncodpers'])['ncodpers'])
final_preds = [" ".join(list(target_cols[pred])) for pred in preds]
out_df = pd.DataFrame({'ncodpers':test_id, 'added_products':final_preds})
out_df.to_csv(outflnm, index=False)
print(datetime.datetime.now()-start_time)frffrrf

In [16]:
out_df

,added_products,ncodpers
0,ind_recibo_ult1 ind_reca_fin_ult1 ind_ctop_fin...,15889
1,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...,1170544
2,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...,1170545
3,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...,1170547
4,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...,1170548
5,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...,1170550
6,ind_tjcr_fin_ult1 ind_nom_pens_ult1 ind_cno_fi...,1170552
7,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...,1170553
8,ind_recibo_ult1 ind_reca_fin_ult1 ind_nom_pens...,1170555
9,ind_recibo_ult1 ind_nom_pens_ult1 ind_nomina_u...,1170557
